Задание 1.

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [ ]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

import time

time.sleep(0.2)


result = pd.DataFrame()

KEYWORDS = ['python', 'парсинг']

request = requests.get('https://habr.com/ru/all/')

request_parsed = BeautifulSoup(request.text, 'html.parser')

posts = request_parsed.find_all('article', class_='post post_preview')


for post in posts:
    post_date = post.find('span', class_='post__time').text
    post_header = post.find('a', class_='post__title_link').text
    post_link = post.find('a').get('href')
    post_preview = post.find('div', class_ = 'post__text').text
    for keyword in KEYWORDS:
        if keyword in post_preview:
            row = {'Дата': post_date, 'Заголовок': post_header, 'Ссылка': post_link}
            result = pd.concat([result, pd.DataFrame([row])]) 
    
    
    
result    

Задание 2.

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [50]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
time.sleep(1)
email_check = ["pbaholdin@mail.ru", "info@avtosdelka.su"]
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {'emailAddresses': email_check}
headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'}
request = requests.post(URL, headers = headers, json = params)
dict_ = json.loads(request.text)

#почта:
#for key in dict_['summary']:
    #print(key)
    
#утечка:
breaches = []
for key in dict_['breaches']:
    breaches.append(key)
    
#источник утечки:
breach_sources = []
for key in dict_['breaches']:
    breach_sources.append(dict_['breaches'][key]['site'])
    
#дата публикации
postdates = []
for key in dict_['breaches']:
    postdates.append(dict_['breaches'][key]['publishDate'])
    
#описание утечки
breach_descriptions = []
for key in dict_['breaches']:
    breach_descriptions.append(dict_['breaches'][key]['description'])
    
    
#соотнесение почты и номера утечки:
email_and_breach_number = {}
for key, value in dict_['summary'].items():
    if key not in email_and_breach_number:
        email_and_breach_number[key] = value['breaches']


parsed_data_dictionary = {'Утечка':breaches, 'Дата публикации утечки':postdates, 'Источник утечки':breach_sources, 'Описание утечки':breach_descriptions}

Result = pd.DataFrame(parsed_data_dictionary)

           
#если номер утечки равен ключу - в столбец "Почта" подставляем значение словаря 
def email_of_the_breach(row):
    for key, value in email_and_breach_number.items():
        for i in value:
            if str(i) == row:
                return key
    
        
Result['Почта'] = Result['Утечка'].apply(email_of_the_breach)

Final_Result = Result.iloc[:, 1:5]

neworder = ['Почта', 'Дата публикации утечки', 'Источник утечки', 'Описание утечки']

Final_Result=Result.reindex(columns=neworder)

Final_Result


,Почта,Дата публикации утечки,Источник утечки,Описание утечки
0,pbaholdin@mail.ru,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
1,pbaholdin@mail.ru,2016-10-26T00:00:00Z,twitter.com,Login credentials for over 32 Million Twitter ...
2,pbaholdin@mail.ru,2020-07-23T00:00:00Z,wattpad.com,"In June 2020, the online writing community Wat..."
3,pbaholdin@mail.ru,2018-12-13T00:00:00Z,houzz.com,"In July 2018, the housing site Houzz was alleg..."
4,pbaholdin@mail.ru,2016-12-15T00:00:00Z,vedomosti.ru,"In May 2012, Russian business newspaper Vedomo..."
5,pbaholdin@mail.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
6,info@avtosdelka.su,2020-12-03T00:00:00Z,chelyabinsk-gid.info,"In November 2020, a collection of over 23,000 ..."
7,pbaholdin@mail.ru,2020-12-10T00:00:00Z,webscript.rusmf,"In November 2020, a collection of over 23,000 ..."
8,pbaholdin@mail.ru,2019-09-12T00:00:00Z,xrumer.winguild.ru,"At an unconfirmed date, Russian audio equipmen..."
9,pbaholdin@mail.ru,2017-01-16T00:00:00Z,fl.ru,"In February 2015, FL.ru's entire site database..."


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
time.sleep(1)
EMAIL = ['xxx@x.ru', 'yyy@y.com']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
addresses = {'emailAddresses': EMAIL}
res = requests.post(url, json=addresses, headers=headers)
df = pd.DataFrame(res.json()['breaches']).T.reset_index() 
df = df[['site', 'description', 'publishDate']]
df['email'] = [', '.join(el.keys()) for el in list(res.json()['data'].values())]

list(res.json()['data'].values())

[{'xxx@x.ru': [{'breachId': 3176,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'xxx@x.ru': [{'breachId': 3,
    'usernameBreached': True,
    'passwordBreached': True}],
  'yyy@y.com': [{'breachId': 3,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'xxx@x.ru': [{'breachId': 2961,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'yyy@y.com': [{'breachId': 17110,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'yyy@y.com': [{'breachId': 2,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'yyy@y.com': [{'breachId': 41,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'yyy@y.com': [{'breachId': 17670,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'yyy@y.com': [{'breachId': 3669,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'yyy@y.com': [{'breachId': 13094,
    'usernameBreached': True,
    'passwordBreached': True}]},
 {'yyy@y.com': [{'breachId': 358